In [1]:
import os
import re
import random

import numpy as np
import pandas as pd

In [2]:
def load_input(input_txt):
    with open(input_txt) as fp:
        lines = fp.read().strip().split('\n')
    delimiter = lines.index('')
    
    # read wires which the value is already known.
    wires = {}
    for line in lines[:delimiter]:
        wire, value = line.split(': ')
        value = int(value)
        wires[wire] = value
        #print(f"{line}: {wire}={value}")
    
    ## load commands.
    commands = []
    for line in lines[delimiter+1:]:
        line_1 = line.split(' -> ')
        assert len(line_1) == 2, print(f"load error1 at: {line}")
        wire_3 = line_1[1]
        
        line_2 = line_1[0].split(' ')
        assert len(line_2) == 3, print(f"load error2 at: {line}")
        wire_1, gate, wire_2 = line_2
        #print(f"{line}: {wire_1} x {wire_2} in {gate} = {wire_3}")
    
        commands.append({'wire_1': wire_1, 'wire_2': wire_2, 'wire_3': wire_3, 'gate': gate})

    return wires, commands

In [3]:
def known_wires(wires):
    return list(wires.keys())

def process(a, b, gate):
    assert gate in ['AND', 'OR', 'XOR'], print("gate should be one of AND / OR / XOR.")
    value = -1
    if gate == 'AND':
        value = a & b
    elif gate == 'OR':
        value = a | b
    elif gate == 'XOR':
        value = a ^ b
    return value

## part 1

In [4]:
def calc_wire_values(wires, commands, iter_max=1000, debug=False):
    i = 0
    while len(commands) > 0:
        commands_ = []
        for command in commands:
            wire_1 = command['wire_1']
            wire_2 = command['wire_2']
            
            if command['wire_1'] in known_wires(wires) and command['wire_2'] in known_wires(wires):
                value_1 = wires[wire_1]
                value_2 = wires[wire_2]
                # update.
                wires[command['wire_3']] = process(value_1, value_2, command['gate'])
            else:
                commands_.append(command)
        commands = commands_
        i += 1
        if i > iter_max:
            break
        if debug:
            print(f'[{i}]: {len(commands_)}')
    return wires


def extract_z(wires):
    df = pd.DataFrame(wires.items(), columns=['wire', 'value'])
    df_ = df[df['wire'].str.contains('z')]
    df_ = df_.sort_values(by=['wire'])
    return df_


def bin2dec(z_bin):
    z = 0
    for i, value in enumerate(z_bin):
        z += 2 ** i * value
        #print(f"{i}, {value}, {z}")
    return z

In [5]:
wires, commands = load_input("input_2.txt")
calc_wire_values(wires, commands, iter_max=1000, debug=False)
df = extract_z(wires)
z_bin = list(df['value'])
z  = bin2dec(z_bin)
print(f"decimal number on the wires starting with z: {z}")

decimal number on the wires starting with z: 2024


## part 2

In [6]:
wires, commands = load_input("input.txt")
df = pd.DataFrame(commands)
n_commands = len(df)
n_wires    = int(len(wires)/2)+1

In [7]:
#i = 0
def _required_wires(wire):
    try:
        df_ = df[df['wire_3'] == wire]
        idx = df_.index.item()
    except:
        return 0
    
    wire_indices.append(idx)
    # i += 1
    # assert i<10
        
    # print(df_)
    # print(wire_idx)
    # print('===')
    
    wire = df_['wire_1'].item()
    _required_wires(wire)
    wire = df_['wire_2'].item()
    _required_wires(wire)


def required_wires(i):
    i_str = str(i).zfill(2)
    _required_wires('z' + i_str)
    return wire_indices

In [8]:
wires_indices = {}
for i in range(46):
    i_str = str(i).zfill(2)
    wire_indices = []
    _required_wires('z' + i_str)
    wire_indices = list(set(wire_indices))
    wire_indices.sort()
    wires_indices[i] = wire_indices
    #print(f"== {i_str} ==")
    #print(wire_indices)

In [9]:
# def occurance(i):
#     n = 0
#     for i_wires in range(n_wires):
#         #print(i_command)
#         if i in wires_indices[i_wires]:
#             n += 1
#             #print(wires_indices[i_command])
#     return n

# for i in range(n_wires):
#     print(f"{i}: {occurance(i)}")

In [10]:
def init_wires(n=7):
    wires = {}
    for i in range(n+1):
        i_str = str(i).zfill(2)
        wires['x' + i_str] = 0
        wires['y' + i_str] = 0
    return dict(sorted(wires.items()))


def dec2bin(x):
    x_bin = list("{0:b}".format(x))
    x_bin.reverse()
    x_bin = [int(d) for d in x_bin]
    return x_bin


def set_xy(x, y, n=7):
    wires = init_wires(n=n)

    x_bin = dec2bin(x)
    for i, d in enumerate(x_bin):
        i_str = str(i).zfill(2)
        wires['x' + i_str] = d
    
    y_bin = dec2bin(y)
    for i, d in enumerate(y_bin):
        i_str = str(i).zfill(2)
        wires['y' + i_str] = d

    return wires


def get_diff_idx(list1, list2):
    diff = []
    i = 0
    for i1, i2 in zip(list1, list2):
        if not i1==i2:
            diff.append(i)
        i += 1
    return diff

#set_xy(43, 24, n=6)

In [11]:
def get_suspects(wires, commands, x, y, n=45):
    # mathematical calculation.
    z_bin = dec2bin(x+y)
    z_bin = z_bin + [0] * (n+1 - len(z_bin))
    #print(z_bin)
    
    # result of the calculator.
    wires = set_xy(x, y, n=n)
    calc_wire_values(wires, commands, iter_max=1000, debug=False)
    df = extract_z(wires)
    z_bin_ = list(df['value'])
    #print(z_bin_)
    
    # difference between mathematical calculation and the result of the calculator.
    indices_diff = []
    if not z_bin == z_bin_:
        indices_diff = get_diff_idx(z_bin, z_bin_) 
    
    # which ones are the suspects?
    suspects = []
    for i in indices_diff:
        suspects.extend(wires_indices[i])
    suspects = list(set(suspects))
    suspects.sort()

    #print(indices_diff)   
    #print(suspects)
    return suspects, indices_diff

In [12]:
#suspects = np.arange(n_commands)
#freq = np.zeros((n_commands)).astype(int)
freq = np.zeros((n_wires)).astype(int)

for i in range(1500):
    #len_suspects_ = len(suspects)
    x = random.randint(0, 2**45)
    y = random.randint(0, 2**45)
    suspects, indices_diff = get_suspects(wires, commands, x, y)
    # suspects  = list(set(suspects) & set(suspects_))
    # suspects.sort()
    for n in indices_diff:
        freq[n] += 1
    #print(indices_diff)
#    if len(suspects) != len_suspects_:
    #print(f"[{i}] {len(suspects)} : {x} - {y}")
#print(f"[{i}] {indices_diff} : {x} - {y}")
    #print(suspects)
print(freq)

[   0    0    0    0    0    0    0    0    0  762  762  373  177   83
   40   18   10    6    5    3 1133 1134  556  268  128   68   35   15
    5    3 1111  585  283  134 1095  911  456  222  109   60   29   19
    9    7    2    0]


In [13]:
# when freq == 0, nothing will affect another places. 
wires_safe = np.where(freq == 0)[0]
commands_safe = set()
for i in wires_safe:
     commands_safe = set(wires_indices[i]) | commands_safe
commands_suspicious = set(np.arange(n_commands)) - commands_safe
len(commands_suspicious)

38

In [14]:
# list(set(wires_indices[9]) 
#      & set(wires_indices[10]) 
#      & set(wires_indices[20])
#      & set(wires_indices[21]) 
#      & set(wires_indices[30])
#      & set(wires_indices[34]) 
#      & set(wires_indices[35])
#     )
# 9-10
# 20-21

#print(wires_indices[20])
#print(wires_indices[21])
#commands_suspicious
wire_indices = []
wire_indices_9 = required_wires(9)
wire_indices = []
wire_indices_10 = required_wires(10)
wire_indices = []
wire_indices_20 = required_wires(20)
wire_indices = []
wire_indices_21 = required_wires(21)
wire_indices = []
wire_indices_30 = required_wires(30)
wire_indices = []
wire_indices_34 = required_wires(34)

print(wire_indices_30)
print(wire_indices_34)

[157, 93, 20, 92, 181, 134, 108, 103, 13, 3, 18, 189, 143, 167, 100, 171, 47, 54, 61, 6, 170, 42, 52, 214, 133, 79, 164, 162, 198, 129, 159, 76, 111, 2, 21, 158, 30, 140, 212, 195, 174, 36, 142, 200, 88, 207, 155, 192, 0, 41, 89, 196, 168, 90, 122, 217, 146, 153, 69, 182, 211, 25, 105, 51, 169, 57, 154, 29, 183, 11, 66, 148, 96, 165, 210, 163, 118, 37, 73, 147, 60, 82, 201, 15, 109, 48, 188, 27, 71, 145, 102, 178, 9, 128, 7, 128, 211, 25, 105, 51, 169, 57, 154, 29, 183, 11, 66, 148, 96, 165, 210, 163, 118, 37, 73, 147, 60, 82, 201, 15, 109, 48, 188, 27, 71, 145, 102, 178, 9, 59, 191, 46, 186, 151, 63, 179, 190, 58, 110, 12, 112, 33, 219, 86, 34, 98, 5, 107, 131, 77, 187]
[138, 62, 202, 55, 101, 175, 221, 218, 35, 120, 19, 156, 180, 193, 187, 93, 20, 92, 181, 134, 108, 103, 13, 3, 18, 189, 143, 167, 100, 171, 47, 54, 61, 6, 170, 42, 52, 214, 133, 79, 164, 162, 198, 129, 159, 76, 111, 2, 21, 158, 30, 140, 212, 195, 174, 36, 142, 200, 88, 207, 155, 192, 0, 41, 89, 196, 168, 90, 122, 217, 

In [15]:
# [20, 21, 22, 23]: 7628113595552 - 30168344363313
# [20, 21, 34, 35]: 3333366260476 - 33391937493479
# [20, 21, 30]: 14441797724076 - 12854979024379
# [20, 21, 30, 34]: 6251231177171 - 30326922175249
# [20, 21, 22, 23, 30, 31, 32]: 3301494794025 - 4751325017906
# [20, 21, 22, 23, 30, 31, 32, 33]: 33211909452755 - 11300451967256
# [20, 21, 22, 23, 30, 31, 34, 35]: 9234551233917 - 5920097924762
# [9, 10, 20, 21, 34, 35, 36, 37, 38, 39, 40, 41]: 14618062164479 - 18366201579556
# [34]: 8478684163876 - 34766440879088
# [34, 35]: 6750277329889 - 14665237685475
x = random.randint(0, 2**45)
y = random.randint(0, 2**45)
_, indices_diff = get_suspects(wires, commands, x, y)
print(f"{indices_diff}: {x} - {y}")

[9, 10, 30, 34, 35, 36]: 18100713724040 - 19625385304879


In [16]:
x = 8478684163876
y = 34766440879088
n = 45
z_bin = dec2bin(x+y)
z_bin = z_bin + [0] * (n+1 - len(z_bin))
print(z_bin)

# result of the calculator.
wires = set_xy(x, y, n=n)
calc_wire_values(wires, commands, iter_max=1000, debug=False)
df = extract_z(wires)
z_bin_ = list(df['value'])
print(z_bin_)

[0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1]
[0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1]


In [17]:
# 208: y09 AND x09 -> z09
# 128: y09 XOR x09 -> fvp
# 157	fmn	kqh	z30	XOR
# 138	bmh	spk	z34	AND
df_commands = pd.DataFrame(commands)
df_commands[df_commands['wire_3'] == 'z34']

#commands_suspicious

,wire_1,wire_2,wire_3,gate
138,bmh,spk,z34,AND
